Notebook to actually create item recommendations for each user and each model:

In [60]:
import pandas as pd
import numpy as np
from dataloading import DataLoader
from datapreparation_py import DataPreparation
from cv_py import CrossValidation
from eda_py import EDA
from implicit.evaluation import ranking_metrics_at_k, train_test_split
from implicit.nearest_neighbours import CosineRecommender
from implicit.als import AlternatingLeastSquares
from eALS_adaptor import eALSAdaptor
from implicit.lmf import LogisticMatrixFactorization
from implicit.bpr import BayesianPersonalizedRanking
import matplotlib.pyplot as plt

%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

C:\Users\781110104\OneDrive - Genpact\Documents\VSCode


In [3]:
dl = DataLoader()
user_item_co = dl.import_data('AGCO', 'CO', 'df')
user_item_co_t = dl.import_data('TEREX', 'CO', 'df')

In [4]:
user_item_filtered = dl.remove_low_interact_items(user_item_co, 1)
user_item_filtered_t = dl.remove_low_interact_items(user_item_co_t, 1)

user_item_filtered_log = dl.log_scale_df(user_item_filtered, 0.01)
user_item_filtered_log_t = dl.log_scale_df(user_item_filtered_t, 0.01)

user_item_csr = dl.to_csr(user_item_filtered_log)
user_item_csr_t = dl.to_csr(user_item_filtered_log_t)

In [65]:
dp_t = DataPreparation(user_item_filtered_log_t)
full_df_t = dp_t._get_full_user_item()

dp = DataPreparation(user_item_filtered_log)
full_df = dp._get_full_user_item()

In [62]:
model_ials_a = AlternatingLeastSquares(factors=150, regularization=90, alpha=0.3, iterations=25)
model_ials_t = AlternatingLeastSquares(factors=100, regularization=90, alpha=1.3, iterations=25)
model_eals_t = eALSAdaptor(factors=50, regularization=50, alpha=0.1, w0=5000, num_iter=25)
model_bpr_a = BayesianPersonalizedRanking(factors=50, regularization=0.03, learning_rate=0.005, iterations=30)
model_lmf_a = LogisticMatrixFactorization(factors=10, regularization=40, iterations=25, neg_prop=50)
model_cosine_a = CosineRecommender()
model_cosine_t = CosineRecommender()

In [63]:
model_ials_a.fit(user_item_csr)
model_ials_t.fit(user_item_csr_t)
model_eals_t.fit(user_item_csr_t)
model_bpr_a.fit(user_item_csr)
model_lmf_a.fit(user_item_csr)
model_cosine_a.fit(user_item_csr)
model_cosine_t.fit(user_item_csr_t)

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

converting type of user_items to <class 'numpy.float32'>


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

c:\Users\781110104\Anaconda3\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.032929420471191406 seconds
  warnings.warn(


  0%|          | 0/202000 [00:00<?, ?it/s]

c:\Users\781110104\Anaconda3\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(


  0%|          | 0/16690 [00:00<?, ?it/s]

In [64]:
def get_recommendations(model, user_item_csr, user_item_full):
    recoms_t = model.recommend(userid=pd.Series(user_item_csr.tocoo().row).unique(), user_items=user_item_csr)

    for u in range(len(recoms_t[0])):
        recos_u = recoms_t[0][u]
        items_u = user_item_full[user_item_full.item_codes.isin(recos_u)]['item'].unique()
        user = user_item_full[user_item_full.user_codes==u]['user'].unique()[0]
        df = pd.DataFrame({user : items_u}).T
        if u == 0:
            ret_df = df
        else:
            ret_df = pd.concat([ret_df, df], axis=0)
    return ret_df

In [66]:
recos_ials_a = get_recommendations(model_ials_a, user_item_csr, full_df)
recos_ials_t = get_recommendations(model_ials_t, user_item_csr_t, full_df_t)
recos_eals_t = get_recommendations(model_eals_t, user_item_csr_t, full_df_t)
recos_bpr_a = get_recommendations(model_bpr_a, user_item_csr, full_df)
recos_lmf_a = get_recommendations(model_lmf_a, user_item_csr, full_df)
recos_cosine_a = get_recommendations(model_cosine_a, user_item_csr, full_df)
recos_cosine_t = get_recommendations(model_cosine_t, user_item_csr_t, full_df_t)

In [69]:
recos_ials_t.head()

,0,1,2,3,4,5,6,7,8,9
AKSEL,02220220,05790360,06630306,06630406,07290361,09130180,10.23.6555,12230735,2212-3008,2531-5152
ALMAHROOS,1270278GT,13037-SGT,25817GT,35586GT,45071GT,45074GT,56302GT,60884GT,66813GT,71376GT
ALVAJAL,01910231_15MTR,06650488,09900405,12090363,12890228,2531-1000,2531-1021,06650486,06650487,12150184
ARACO,2402-2003,2495-3003,02020211,05110105,05110107,05120115,12230735,2210-0023,2211-0010,N1322802000000Z
ASCENDUM,1210707018,1290707014,5050658553,5253100003,5253100544,5368632611,5388660778,5399000725,5468660575,5522655650
